CREATE database platzi_operation;

CREATE DATABASE IF NOT EXISTS platzi_operation;

USE platzi_operation;

SHOW TABLES;

```mysql
SHOW WARNINGS;
DESCRIBE table;
SHOW FULL COLUMNS FROM table;
SELECT COUNT(*) FROM table;

-- Listar todos los autores con ID entre 1 y 5 con los filtro mayor y menor igual
SELECT * FROM authors WHERE author_id > 0 AND author_id <= 5;

-- Listar todos los autores con ID entre 1 y 5 con el filtro BETWEEN
SELECT * FROM authors WHERE author_id BETWEEN 1 AND 5;

-- Listar los libros con filtro de author_id entre 1 y 5
SELECT book_id, author_id, title FROM books WHERE author_id BETWEEN 1 AND 5;

-- Listar nombre y titulo de libros mediante el JOIN de las tablas books y authors
SELECT b.book_id, a.name, a.author_id, b.title
FROM books AS b
JOIN authors AS a
  ON a.author_id = b.author_id
WHERE a.author_id BETWEEN 1 AND 5;

-- Listar transactions con detalle de nombre, titulo y tipo. Con los filtro genero = F y tipo = Vendido.
-- Haciendo join entre transactions, books y clients.
SELECT c.name, b.title, t.type
FROM transactions AS t
JOIN books AS b
  ON t.book_id = b.book_id
JOIN clients AS c
  ON t.client_id = c.client_id
WHERE c.gender = 'F'
  AND t.type = 'sell';

-- Listar transactions con detalle de nombre, titulo, autoor y tipo. Con los filtro genero = M y de tipo = Vendido y Devuelto.
-- Haciendo join entre transactions, books, clients y authors.
SELECT c.name, b.title, a.name, t.type
FROM transactions AS t
JOIN books AS b
  ON t.book_id = b.book_id
JOIN clients AS c
  ON t.client_id = c.client_id
JOIN authors AS a
  ON b.author_id = a.author_id
WHERE c.gender = 'M'
  AND t.type IN ('sell', 'lend');
```

--------------------------------------------

Para novakorp necesito crear tablas, no dbs.

``` mysql
CREATE TABLE IF NOT EXISTS books(
    book_id INTEGER UNSIGNED PRIMARY KEY AUTO_INCREMENT,
    author_id INTEGER UNSIGNED ,
    title VARCHAR(100) NOT NULL,
    `year` INTEGER UNSIGNED NOT NULL DEFAULT 1900,
    `language` VARCHAR(2) NOT NULL DEFAULT 'es' COMMENT 'ISO 639-1 Language',
    cover_url VARCHAR(500),
    price DOUBLE(6,2) NOT NULL DEFAULT 10.0,
    selleable TINYINT(1) DEFAULT 1,
    copies INTEGER NOT NULL DEFAULT 1,
    description TEXT,
);
```

``` mysql
CREATE TABLE IF NOT EXISTS author(
    author_id INTEGER UNSIGNED PRIMARY KEY AUTO_INCREMENT,
    `name` VARCHAR(100) NOT NULL,
    nationality VARCHAR(3)
);
```

``` mysql
CREATE TABLE IF NOT EXISTS clients(
    client_id INTEGER UNSIGNED PRIMARY KEY AUTO_INCREMENT,
    `name` VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL UNIQUE,
    birthdate DATETIME,
    gender ENUM('M', 'F', 'ND') NOT NULL,
    active TINYINT(1) NOT NULL DEFAULT 1,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        ON UPDATE CURRENT_TIMESTAMP
);
```

``` mysql
CREATE TABLE IF NOT EXISTS operations(
    operation_id INTEGER UNSIGNED PRIMARY KEY AUTO_INCREMENT,
    book_id INTEGER UNSIGNED FOREIGN KEY,
    client_id INTEGER UNSIGNED FOREIGN KEY,
    `type` ENUM('Prestado', 'Vendido', 'Devuelto') NOT NULL,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        ON UPDATE CURRENT_TIMESTAMP,
    finished TINYINT(1) NOT NULL,
);
```

--------------------------------------------

``` mysql
-- Uso del JOIN implícito
SELECT b.title, a.name
FROM authors AS a, books AS b
WHERE a.author_id = b.author_id
LIMIT 10;

-- Uso del JOIN explícito
SELECT b.title, a.name
FROM books AS b
INNER JOIN authors AS a
  ON a.author_id = b.author_id
LIMIT 10;

--  JOIN y order by (por defecto es ASC)
SELECT a.author_id, a.name, a.nationality, b.title
FROM authors AS a
JOIN books AS b
  ON b.author_id = a.author_id
WHERE a.author_id BETWEEN 1 AND 5
ORDER BY a.author_id DESC;

-- LEFT JOIN para traer datos incluso que no existen, como el caso del author_id = 4 que no tene ningún libro registrado.
SELECT a.author_id, a.name, a.nationality, b.title
FROM authors AS a
LEFT JOIN books AS b
  ON b.author_id = a.author_id
WHERE a.author_id BETWEEN 1 AND 5
ORDER BY a.author_id;

-- Contar número de libros tiene un autor.
-- Con COUNT (contar), es necesario tener un GROUP BY (agrupado por un criterio)
SELECT a.author_id, a.name, a.nationality, COUNT(b.book_id)
FROM authors AS a
LEFT JOIN books AS b
  ON b.author_id = a.author_id
WHERE a.author_id BETWEEN 1 AND 5
GROUP BY a.author_id
ORDER BY a.author_id;
```

----------------------------------
``` mysql
--1. ¿Qué nacionalidades hay?  
-- Mediante la clausula DISTINCT trae solo los elementos distintos

SELECT DISTINCT nationality 
FROM authors
ORDER BY 1;

-- 2. ¿Cuántos escritores hay de cada nacionalidad?
-- IS NOT NULL para traer solo los valores diferentes de nulo
-- NOT IN para traer valores que no sean los declarados (RUS y AUT)

SELECT nationality, COUNT(author_id) AS c_authors
FROM authors
WHERE nationality IS NOT NULL
	AND nationality NOT IN ('RUS','AUT')
GROUP BY nationality
ORDER BY c_authors DESC, nationality ASC;


-- 4. ¿Cuál es el promedio/desviación standard del precio de libros?

SELECT a.nationality,  
  AVG(b.price) AS promedio, 
  STDDEV(b.price) AS std 
FROM books AS b
JOIN authors AS a
  ON a.author_id = b.author_id
GROUP BY a.nationality
ORDER BY promedio DESC;

-- 5. ¿Cuál es el promedio/desviación standard del precio de libros por nacionalidad?
-- Agrupar por la columna pivot

SELECT a.nationality,
  COUNT(b.book_id) AS libros,  
  AVG(b.price) AS promedio, 
  STDDEV(b.price) AS std 
FROM books AS b
JOIN authors AS a
  ON a.author_id = b.author_id
GROUP BY a.nationality
ORDER BY libros DESC;

-- 6. ¿Cuál es el precio máximo/mínimo de un libro?

SELECT nationality, MAX(price), MIN(price)
FROM books AS b
JOIN authors AS a
  ON a.author_id = b.author_id
GROUP BY nationality;

-- 7. ¿cómo quedaría el reporte de préstamos?
-- CONCAT: para concatenar en cadenas de texto.
-- TO_DAYS: recibe un timestamp ó un datetime

SELECT c.name, t.type, b.title, 
  CONCAT(a.name, " (", a.nationality, ")") AS autor,
  TO_DAYS(NOW()) - TO_DAYS(t.created_at)
FROM transactions AS t
LEFT JOIN clients AS c
  ON c.client_id = t.client_id
LEFT JOIN books AS b
  ON b.book_id = t.book_id
LEFT JOIN authors AS a
  ON b.author_id = a.author_id;


-- SUM(), para sumar cada valor(1) en una tupla
SELECT 
  COUNT(book_id), 
  SUM(IF(year < 1950, 1, 0)) AS '<1950',
  SUM(IF(year >= 1950 AND year < 1990, 1, 0)) AS '<1990',
  SUM(IF(year >= 1990 AND year < 2000, 1, 0)) AS '<2000',
  SUM(IF(year >= 2000, 1, 0)) AS '<hoy'
FROM books;

-- Agrupar el query anterior y mostrar su nacionalidad
SELECT 
  nationality,
  COUNT(book_id), 
  SUM(IF(year < 1950, 1, 0)) AS '<1950',
  SUM(IF(year >= 1950 AND year < 1990, 1, 0)) AS '<1990',
  SUM(IF(year >= 1990 AND year < 2000, 1, 0)) AS '<2000',
  SUM(IF(year >= 2000, 1, 0)) AS '<hoy'
FROM books AS b
JOIN authors AS a
  ON a.author_id = b.author_id
WHERE a.nationality IS NOT NULL
GROUP BY nationality;
```


¿Qué es una base de datos?

Es un lugar donde podemos ir almacenando datos puntuales de cualquier cantidad de cosas para después operar sobre esos datos y convertirlos en información. Esa información convertirla en operaciones de negocio y las operaciones de negocio convertirlas en dinero, crecimiento sabiduría lo que sea. TODO RESIDE EN LOS DATOS Y CÓMO OPERAMOS SOBRE LOS DATOS  

Se llama base de datos, o también banco de datos, a un conjunto de información perteneciente a un mismo contexto, ordenada de modo sistemático para su posterior recuperación, análisis y/o transmisión. Existen actualmente muchas formas de bases de datos, que van desde una biblioteca hasta los vastos conjuntos de datos de usuarios de una empresa de telecomunicaciones. 

Las bases de datos son el producto de la necesidad humana de almacenar la información, es decir, de preservarla contra el tiempo y el deterioro, para poder acudir a ella posteriormente. En ese sentido, la aparición de la electrónica y la computación brindó el elemento digital indispensable para almacenar enormes cantidades de datos en espacios físicos limitados, gracias a su conversión en señales eléctricas o magnéticas.  

El manejo de las bases de datos se lleva mediante sistemas de gestión (llamados DBMS por sus siglas en inglés: Database Management Systems o Sistemas de Gestión de Bases de Datos), actualmente digitales y automatizados, que permiten el almacenamiento ordenado y la rápida recuperación de la información. En esta tecnología se halla el principio mismo de la informática. En la conformación de una base de datos se pueden seguir diferentes modelos y paradigmas, cada uno dotado de características, ventajas y dificultades, haciendo énfasis en su estructura organizacional, su jerarquía, su capacidad de transmisión o de interrelación, etc. Esto se conoce como modelos de base de datos y permite el diseño y la implementación de algoritmos y otros mecanismos lógicos de gestión, según sea el caso específico.

- SHOW databases; - muestra las bases de datos existentes.
- USE database_name; - selecciona una base de datos específica.
- SHOW tables; - muestras las tablas de la base de datos.
- SELECT database(); - me muestra el nombre de la base de datos seleccionada.
- CREATE database database_name; - crea una nueva base de datos.
- CREATE DATABASE IF NOT EXISTS database_name; - crea una base de datos si no existe.
- SHOW warnings; - muestra las advertencias.
- DROP table table_name; - Elimina permanentemente una tabla.
- DESCRIBE table_name; - Nos indica las columnas que tenemos en una tabla.
- SHOW FULL COLUMNS FROM table_name; - es parecido al comando DESCRIBE pero muestra mas datos.
- INSERT INTO table_name(columns) VALUES(values); - inserta una tupla.
- ON DUPLICATE KEY IGNORE ALL - esta sentencia ignora las resticciones al insertar una tupla
con un valor repetido y que esta restringido en una columna con UNIQUE (Nota: nunca utilizarlo).
- ON DUPLICATE KEY UPDATE column = VALUES(value) - al insertar una tupla con un campo duplicado
actualiza un el valor de un campo específico con un nuevo valor tomado de los datos insertados.
- SELECT * FROM table_name WHERE column_value = 1\G - en lugar de cerrar la sentencia con ;
se utiliza \G, lo cual muestra los datos de una manera mas legible.
- mysql -u root -p < all_schema.sql - con este comando podemos ejecutar un script SQL inmediatamente
despues de acceder a la base de datos.
- mysql -u root -p -D database_name < all_schema.sql - este comando es parecido al anterior solo
que con la bandera -D indicamos el nombre de la base de datos sobre la que queremos ejecutar el script.
- SELECT YEAR(NOW()); - esta sentencia me muestra el año de la fecha actual utilizando las funciones YEAR() y NOW().
- SELECT * FROM table_name WHERE column_value like ‘%value%’; - esta sentencia nos muestra las tuplas que en un
campo específico contengan un valor, el wildcard % indica que no nos importa que valor existan antes o despues del
dato que especificamos.
- SELECT COUNT(*) FROM table_name; - devuelve el número de tuplas de una tabla.
- SELECT * FROM table_name WHERE column_value BETWEEN value AND value; - nos devuelve las tuplas que se encuentren
en medio de los valores indicados.
- DELETE FROM table_name WHERE column_value = value; - elimina una tupla de una tabla.
- UPDATE table_name SET [column_value = value, …] WHERE column_value = value; - actualiza una tupla de una tabla.
- TRUNCATE table_name; - borra todo el contenido de una tabla.
- mysqldump -u user -p database_name > esquema.sql - guarda el esquema de una base de datos con todo y datos en un
archivo sql.
- mysqldump -u user -p -d database_name es parecido al comando anterior solo que aquí no se guardan los datos.